In [1]:
import requests

%store -r CLIENT_ID
%store -r API_KEY

ENDPOINT = "https://api.yelp.com/v3/businesses/search"

HEADERS = {'Authorization': 'bearer %s' % API_KEY}

PARAMETERS = {'term': 'restaurants',
              'offset': 0,
              'limit': 50,
              'radius': 40000,
              'location': 'Vancouver, BC'}

response = requests.get(url=ENDPOINT, params=PARAMETERS, headers=HEADERS)

In [2]:
response

<Response [200]>

In [3]:
# open categories json file
import json

with open('Categories.json') as f:
    data = json.load(f)

restaurants = [place for place in data if 'restaurants' in place['parents']]
restaurant_aliases = [restaurant['alias'] for restaurant in restaurants]
restaurant_titles = [restaurant['title'] for restaurant in restaurants]

print("Restaurant aliases: {} || Num: {}".format(restaurant_aliases[:3], len(restaurant_aliases)))
print("Restaurant titles: {} || Num: {}".format(restaurant_titles[:3], len(restaurant_titles)))

Restaurant aliases: ['afghani', 'african', 'andalusian'] || Num: 192
Restaurant titles: ['Afghan', 'African', 'Andalusian'] || Num: 192


In [4]:
import time

PARAMETERS = {'term': 'restaurants',
              'offset': 0, # start at 0
              'limit': 50, # maximum is 50
              'radius': 40000, # in m
              'location': 'Vancouver, BC'}

restaurants_in_vancouver = []

# Cycle through categories
for category in restaurant_aliases:
    PARAMETERS['categories'] = category
    # Cycle through restaurants
    for offset_number in range(0,1000,50):
        PARAMETERS['offset'] = offset_number

        response = requests.get(url=ENDPOINT, params=PARAMETERS, headers=HEADERS)

        if not response.json().get('businesses', False):
            break

        restaurants_in_vancouver.extend(response.json()['businesses'])

        print("{}: {}-{}".format(category, offset_number, offset_number+50))
        
        time.sleep(0.5) ## Don't want to get blocked by Yelp API

afghani: 0-50
african: 0-50
arabian: 0-50
asianfusion: 0-50
asianfusion: 50-100
asianfusion: 100-150
australian: 0-50
austrian: 0-50
bangladeshi: 0-50
bbq: 0-50
bbq: 50-100
belgian: 0-50
bistros: 0-50
brasseries: 0-50
brazilian: 0-50
breakfast_brunch: 0-50
breakfast_brunch: 50-100
breakfast_brunch: 100-150
breakfast_brunch: 150-200
british: 0-50
buffets: 0-50
burgers: 0-50
burgers: 50-100
burgers: 100-150
burgers: 150-200
burmese: 0-50
cafes: 0-50
cafes: 50-100
cafes: 100-150
cafes: 150-200
cajun: 0-50
cambodian: 0-50
caribbean: 0-50
cheesesteaks: 0-50
chicken_wings: 0-50
chicken_wings: 50-100
chickenshop: 0-50
chinese: 0-50
chinese: 50-100
chinese: 100-150
chinese: 150-200
comfortfood: 0-50
creperies: 0-50
delis: 0-50
delis: 50-100
diners: 0-50
dinnertheater: 0-50
dumplings: 0-50
ethiopian: 0-50
filipino: 0-50
fishnchips: 0-50
food_court: 0-50
foodstands: 0-50
french: 0-50
gastropubs: 0-50
german: 0-50
gluten_free: 0-50
greek: 0-50
guamanian: 0-50
halal: 0-50
hawaiian: 0-50
himalayan:

In [5]:
# This number includes duplicates
print(len(restaurants_in_vancouver))

4893


In [7]:
restaurants_file =  open("vancouver_restaurants_duplicates.json", "w")
json.dump(restaurants_in_vancouver, restaurants_file, indent=6)
restaurants_file.close()

In [8]:
# Remove the duplicate entries
res_list = [i for n, i in enumerate(restaurants_in_vancouver) if i not in restaurants_in_vancouver[n + 1:]]

In [9]:
restaurants_file = open("vancouver_restaurants.json", "w")
json.dump(res_list, restaurants_file, indent=6)
restaurants_file.close()

In [10]:
newlist = sorted(res_list, key=lambda k: k['name'])

In [11]:
len(newlist)

3268